In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as spc

In [2]:
######################################### Constants taken from scipy.constants ###################################

# all in SI units
G = spc.G
c = spc.c
hbar = spc.hbar
k = spc.k
mp = spc.proton_mass
me = spc.electron_mass
sigma_sb = spc.sigma

########################################### Definfing constants required #########################################

Msun = 1.989e30
a = 4*sigma_sb/c
gamma = 5.0/3.0 #The adiabatic index has a value γ = 5/3 for simple atomic gases and fully ionized gases.
'''
check: is gamma value correct?
'''
X, Y, Z = 0.734, 0.250, 0.016 #Values taken from Foundations of Astrophysics.
'''
check X, Y, Z values
'''
mu = 2*X + 0.75*Y + 0.5*Z

class star(object):
    def __init__(self, r0, rho_c, T_c, step_size):

        '''
        Given r0, rho_c, T_c calculate M0 and L0
        M0 - Initial Mass
        L0 - Initial Luminosity
        tau0 - Initial optical depth
        '''

        self.M0 = (4.0/3.0)*np.pi*r0**3*rho_c
        self.L0 = self.M0*self.epsilon(rho_c, T_c) 
        self.tau0 = self.kappa(rho_c,T_c)*rho_c
        self.r0 = r0
        self.rho_c = rho_c
        self.T_c = T_c
        self.step_size = step_size
        
        
################################################ Below are DEs required to solve ###################################

    def dpdr(self, r, rho, T, M, L):

        '''
        Hydrostatic equilibrium in rho
        '''

        return -(G*M*rho/r**2 + self.dPdT(rho, T)*self.dTdr(r, rho, T, M, L))/self.dPdp(rho, T)


    def dTdr(self, r, rho, T, M, L):

        '''
        Equation for energy transport. Minimum of energy transport
        due to convection and radiation
        '''

        dTdr_rad = 3*self.kappa(rho, T)*L/(16*np.pi*a*c*T**3*r**2)
        dTdr_conv = (1 - 1/gamma)*T*G*M*rho/(self.P(rho, T)*r**2)

        return -min(dTdr_rad , dTdr_conv)


    def dMdr(self, r, rho):

        '''
        definition of enclosed mass
        '''

        return 4*np.pi*r**2*rho

    def dLdr(self, r, rho, T):

        '''
        Energy generation equation
        '''

        return 4*np.pi*r**2*rho*self.epsilon(rho, T)

    def dtaudr(self, rho, T):

        '''
        Optical Depth
        '''

        return self.kappa(rho, T)*rho


############################################# Functions to assist in solving DEs ###################################

    def epsilon(self, rho, T):
        '''
        Sum up energy generation due to proton-proton chain and CNO cycle
        '''

        rho5 = rho/1e5
        T6 = T/1e6
        Xcno = 0.03*X

        epp = 1.07e-7*rho5*X**2*T6**4
        ecno = 8.24e-26*rho5*X*Xcno*T6**(19.9)
        
        return epp + ecno



    def dPdT(self, rho, T):

        '''
        Differentiating Pressure equation(eq - 5) wrt T and
        considering rho a constant
        '''

        dPdT_ig = rho*k/(mu*mp)
        dPdT_pg = (4.0/3.0)*a*T**3

        return  dPdT_ig + dPdT_pg



    def kappa(self, rho, T):

        '''
        Rosseland Mean Opacities.
        '''
        rho3 = rho/1e3

        Kes = 0.02*(1+X)
        Kff = 1e24*(Z+0.0001)*rho3**(0.7)*T**(-3.5)
        KH = 2.5e-32*(Z/0.02)*rho3**(0.5)*T**9

        '''
        Check: below I return eq 14, but the comments below the equation 14 in the
        project description say "near and below the net opacity is the minimum of that due to free-free/scattering
        and that due to H". I am not entirely sure if I need do put a conditionality here.
        '''

        return ((1/KH) + (1/max(Kes, Kff)))**(-1)

    def P(self, rho, T):

        '''
        Pressure. Sum of non relativistic degenerate gas,
        ideal gas and photon gas
        '''

        P_nr = (((3*np.pi**2)**(2.0/3.0))*(hbar**(2))*((rho/mp)**(5/3)))/(5*me)
        P_ig = rho*k*T/(mu*mp)
        P_pg = (1.0/3.0)*a*T**4

        return P_nr + P_ig + P_pg

    def dPdp(self, rho, T):

        '''
        Differentiating Pressure equation(eq - 5) wrt rho(p) and
        considering T a constant
        '''

        dPdp_nr = (((3*np.pi**2)**(2.0/3.0))*(hbar**2)*((rho/mp)**(2.0/3.0)))/(3.0*mp*me)
        dPdp_ig = k*T/(mu*mp)

        return dPdp_nr + dPdp_ig

################################################### Testing Radius Limit ######################################

    def deltau(self, r, rho, T, M, L):

        '''
        Opacity limit - to ensure that the radius is far enough
        '''

        return self.kappa(rho, T)*rho**2/abs(self.dpdr(r, rho, T, M, L))
    
    def tau_lim(self, r, rho, T, M, L):
        
        '''
        The runge kutta method stops when tau is satisfied
        '''
        
        deltau = self.deltau(r, rho, T, M, L)
        
        if deltau < 1e-23:
            return True
        
        '''
        introducing mass limit, since some fully radiative trial 
        solutions can erroneously extend to very large radii
        '''
        
        if M > 1e3*Msun:
            return True
        
        else:
            print('deltau', deltau)
            print('M', M)
            return False
    
    
################################################### Solving DEs ###################################################
    
    def runge_kutta(self, y0, r0, h, f):
    
        '''
        r0 = initial r value
        y0 = y(r0) = initial y value
        h = 'step size'
        f = 'function'
        '''
        
        #print(y0, r0)
    
        k1 = f(y0,r0)
        k2 = f(y0 + 0.5*h*k1, r0 + 0.5*h)
        k3 = f(y0 + 0.5*h*k2, r0 + 0.5*h)
        k4 = f(y0 + h*k3, r0 + h)

        return y0 + (k1 + 2*k2 + 2*k3 + k4)*h/6, r0+h
    
    def f(self, y0, r0):
        
        '''
        This function takes the y0, r0, values given in the runge kutta method
        and calculates the dpde, dTdr, dMdr, dLdr, dtaudr values.
        '''
        
        rho = y0[0]
        T = y0[1]
        M = y0[2]
        L = y0[3]
        tau = y0[4]
        
        rho_fun = self.dpdr(r0, rho, T, M, L)
        T_fun = self.dTdr(r0, rho, T, M, L)
        M_fun = self.dMdr(r0, rho)
        L_fun = self.dLdr(r0, rho, T)
        tau_fun = self.dtaudr(rho, T)
        
        #print('function array', np.array([rho_fun, T_fun, M_fun, L_fun, tau_fun]))
        
        return np.array([rho_fun, T_fun, M_fun, L_fun, tau_fun])
        
    def solve_DE(self):
        
        rho_vals, T_vals, M_vals, L_vals, tau_vals = [], [], [], [], []
        
        r0 = self.r0
        init_vals = np.array([self.rho_c, self.T_c, self.M0, self.L0, self.tau0])
        
        func_vals, r = self.runge_kutta(init_vals, r0, self.step_size, self.f)
        
        #updating r0 and init values
        r0 = r
        init_vals = func_vals
        
        rho_vals.append(func_vals[0])
        T_vals.append(func_vals[1])
        M_vals.append(func_vals[2])
        L_vals.append(func_vals[3])
        tau_vals.append(func_vals[4])
        
        '''
        Unless the tau limit criteria is met the loop runs
        '''
    
        while star_func.tau_lim(r, func_vals[0], func_vals[1], func_vals[2], func_vals[3]) == False: 
            
            func_vals, r = self.runge_kutta(init_vals, r0, self.step_size, self.f)
            
            rho_vals.append(func_vals[0])
            T_vals.append(func_vals[1])
            M_vals.append(func_vals[2])
            L_vals.append(func_vals[3])
            tau_vals.append(func_vals[4])
    
            #updating r0 and init values
            init_vals = func_vals
            r0 = r
        
        
        return rho_vals, T_vals, M_vals, L_vals, tau_vals  

In [3]:
star_func = star(1e-50, 1, 1, 1e2)
rho, T, M, L, tau = star_func.solve_DE()

deltau 3.739605670596489e-19
M 4188790.204785859
deltau 1.8698028352974236e-19
M 33510321.63828306
deltau 1.2465352235307796e-19
M 113097335.52917753
deltau 9.349014176472147e-20
M 268082573.10610414
deltau 7.479211341168792e-20
M 523598775.5976212
deltau 6.232676117631572e-20
M 904778684.2321851
deltau 5.342293815103573e-20
M 1436755040.2381248
deltau 4.674507088206339e-20
M 2144660584.843616
deltau 4.1551174117296084e-20
M 3053628059.276656
deltau 3.7396056705472353e-20
M 4188790204.7650385
deltau 3.3996415186698473e-20
M 5575279762.536326
deltau 3.1163380587712e-20
M 7238229473.817829
deltau 2.8766197465485035e-20
M 9202772079.836575
deltau 2.6711469074997708e-20
M 11494040321.819284
deltau 2.4930704469902094e-20
M 14137166940.99235
deltau 2.3372535440437243e-20
M 17157284678.581804
deltau 2.1997680414433004e-20
M 20579526275.813297
deltau 2.07755870579793e-20
M 24429024473.912075
deltau 1.968213510746287e-20
M 28730912014.102943
deltau 1.8698028351993126e-20
M 33510321637.610256
de

M 50303135358744.65
deltau 1.625915497561831e-21
M 50965010284845.195
deltau 1.618876902235433e-21
M 51632665741376.625
deltau 1.6118989844545254e-21
M 52306126861079.5
deltau 1.604980962963157e-21
M 52985418776694.36
deltau 1.5981220698601805e-21
M 53670566620961.74
deltau 1.5913215503151056e-21
M 54361595526622.18
deltau 1.5845786622911796e-21
M 55058530626416.2
deltau 1.5778926762754758e-21
M 55761397053084.336
deltau 1.5712628750157909e-21
M 56470219939367.086
deltau 1.5646885532641405e-21
M 57185024418004.97
deltau 1.5581690175266741e-21
M 57905835621738.49
deltau 1.5517035858198132e-21
M 58632678683308.15
deltau 1.5452915874324341e-21
M 59365578735454.42
deltau 1.5389323626939264e-21
M 60104560910917.805
deltau 1.5326252627479508e-21
M 60849650342438.77
deltau 1.5263696493317425e-21
M 61600872162757.805
deltau 1.5201648945607918e-21
M 62358251504615.36
deltau 1.5140103807187592e-21
M 63121813500751.91
deltau 1.507905500052473e-21
M 63891583283907.89
deltau 1.501849654571863e-21
M

M 454614979334653.0
deltau 7.823442588734073e-22
M 457480191366497.56
deltau 7.807109722214671e-22
M 460357416848185.25
deltau 7.7908449093037005e-22
M 463246680912454.3
deltau 7.774647725550385e-22
M 466148008692043.06
deltau 7.7585177500263565e-22
M 469061425319689.8
deltau 7.742454565289192e-22
M 471986955928132.9
deltau 7.726457757346423e-22
M 474924625650110.44
deltau 7.710526915619927e-22
M 477874459618360.75
deltau 7.694661632910842e-22
M 480836482965622.06
deltau 7.678861505364854e-22
M 483810720824632.6
deltau 7.663126132437927e-22
M 486797198328130.6
deltau 7.647455116862456e-22
M 489795940608854.25
deltau 7.631848064613844e-22
M 492806972799541.75
deltau 7.616304584877464e-22
M 495830320032931.25
deltau 7.60082429001605e-22
M 498866007441760.9
deltau 7.585406795537454e-22
M 501914060158768.8
deltau 7.570051720062847e-22
M 504974503316693.2
deltau 7.554758685295222e-22
M 508047362048272.1
deltau 7.539527315988372e-22
M 511132661486243.75
deltau 7.524357239916177e-22
M 5142304

deltau 5.201120187086039e-22
M 1556952162590204.5
deltau 5.193896408058386e-22
M 1563457525233857.2
deltau 5.186692667251351e-22
M 1569980983449608.2
deltau 5.179508881403627e-22
M 1576522562370192.0
deltau 5.172344967714556e-22
M 1583082287128343.0
deltau 5.165200843840936e-22
M 1589660182856796.0
deltau 5.158076427893877e-22
M 1596256274688285.5
deltau 5.150971638435665e-22
M 1602870587755546.0
deltau 5.143886394476664e-22
M 1609503147191312.0
deltau 5.1368206154722345e-22
M 1616153978128318.0
deltau 5.12977422131968e-22
M 1622823105699298.5
deltau 5.122747132355229e-22
M 1629510555036988.0
deltau 5.11573926935102e-22
M 1636216351274121.0
deltau 5.108750553512135e-22
M 1642940519543432.0
deltau 5.1017809064736435e-22
M 1649683084977655.2
deltau 5.094830250297668e-22
M 1656444072709525.0
deltau 5.08789850747048e-22
M 1663223507871775.8
deltau 5.080985600899632e-22
M 1670021415597142.0
deltau 5.074091453911077e-22
M 1676837821018357.8
deltau 5.067215990246352e-22
M 1683672749268157.8
d

deltau 3.8873235053728193e-22
M 3729183938210362.0
deltau 3.8832868236916758e-22
M 3740825502622534.0
deltau 3.879258516867765e-22
M 3752491269860713.0
deltau 3.8752385588652603e-22
M 3764181265057629.0
deltau 3.871226923756144e-22
M 3775895513346011.0
deltau 3.867223585719653e-22
M 3787634039858588.5
deltau 3.8632285190417154e-22
M 3799396869728090.5
deltau 3.859241698114416e-22
M 3811184028087246.5
deltau 3.8552630974354283e-22
M 3822995540068786.0
deltau 3.851292691607489e-22
M 3834831430805438.0
deltau 3.8473304553378453e-22
M 3846691725429931.5
deltau 3.8433763634377288e-22
M 3858576449074996.0
deltau 3.839430390821809e-22
M 3870485626873360.5
deltau 3.835492512507678e-22
M 3882419283957754.0
deltau 3.8315627036153065e-22
M 3894377445460906.0
deltau 3.8276409393665316e-22
M 3906360136515545.5
deltau 3.8237271950845297e-22
M 3918367382254401.5
deltau 3.819821446193311e-22
M 3930399207810203.0
deltau 3.815923668217185e-22
M 3942455638315679.0
deltau 3.8120338367802656e-22
M 39545366

...

M 2.010084335849305e+20
deltau 1.0288521276245138e-23
M 2.0102502728212627e+20
deltau 1.0288238074409237e-23
M 2.0104162189246703e+20
deltau 1.0287954888155554e-23
M 2.0105821741597786e+20
deltau 1.0287671717482808e-23
M 2.010748138526839e+20
deltau 1.0287388562389715e-23
M 2.0109141120261023e+20
deltau 1.0287105422874979e-23
M 2.01108009465782e+20
deltau 1.028682229893732e-23
M 2.0112460864222432e+20
deltau 1.0286539190575453e-23
M 2.0114120873196228e+20
deltau 1.028625609778809e-23
M 2.01157809735021e+20
deltau 1.0285973020573953e-23
M 2.0117441165142563e+20
deltau 1.0285689958931743e-23
M 2.0119101448120127e+20
deltau 1.0285406912860191e-23
M 2.01207618224373e+20
deltau 1.0285123882358e-23
M 2.01224222880966e+20
deltau 1.0284840867423885e-23
M 2.0124082845100533e+20
deltau 1.0284557868056569e-23
M 2.0125743493451615e+20
deltau 1.0284274884254761e-23
M 2.0127404233152355e+20
deltau 1.0283991916017183e-23
M 2.0129065064205264e+20
deltau 1.0283708963342545e-23
M 2.0130725986612855e+20


deltau 1.0218202526766042e-23
M 2.0520216690908122e+20
deltau 1.0217923180812649e-23
M 2.052189905936717e+20
deltau 1.0217643850124266e-23
M 2.052358151977112e+20
deltau 1.021736453469966e-23
M 2.0525264072122486e+20
deltau 1.0217085234537567e-23
M 2.0526946716423776e+20
deltau 1.0216805949636742e-23
M 2.0528629452677503e+20
deltau 1.0216526679995932e-23
M 2.053031228088618e+20
deltau 1.0216247425613888e-23
M 2.0531995201052315e+20
deltau 1.0215968186489352e-23
M 2.0533678213178422e+20
deltau 1.0215688962621087e-23
M 2.0535361317267014e+20
deltau 1.0215409754007833e-23
M 2.05370445133206e+20
deltau 1.0215130560648335e-23
M 2.0538727801341693e+20
deltau 1.0214851382541352e-23
M 2.0540411181332804e+20
deltau 1.0214572219685635e-23
M 2.0542094653296447e+20
deltau 1.0214293072079926e-23
M 2.054377821723513e+20
deltau 1.0214013939722979e-23
M 2.0545461873151364e+20
deltau 1.0213734822613545e-23
M 2.0547145621047663e+20
deltau 1.0213455720750375e-23
M 2.054882946092654e+20
deltau 1.021317663

deltau 1.0145256742157261e-23
M 2.09658549015441e+20
deltau 1.0144981368194837e-23
M 2.0967561538043245e+20
deltau 1.0144706009172967e-23
M 2.096926826714783e+20
deltau 1.014443066509042e-23
M 2.0970975088860368e+20
deltau 1.0144155335945995e-23
M 2.097268200318337e+20
deltau 1.0143880021738477e-23
M 2.0974389010119347e+20
deltau 1.0143604722466642e-23
M 2.0976096109670813e+20
deltau 1.0143329438129276e-23
M 2.0977803301840277e+20
deltau 1.0143054168725169e-23
M 2.0979510586630252e+20
deltau 1.0142778914253096e-23
M 2.098121796404325e+20
deltau 1.0142503674711853e-23
M 2.098292543408178e+20
deltau 1.0142228450100217e-23
M 2.098463299674836e+20
deltau 1.0141953240416978e-23
M 2.098634065204549e+20
deltau 1.0141678045660916e-23
M 2.098804839997569e+20
deltau 1.0141402865830825e-23
M 2.0989756240541473e+20
deltau 1.0141127700925482e-23
M 2.0991464173745347e+20
deltau 1.0140852550943672e-23
M 2.0993172199589822e+20
deltau 1.0140577415884183e-23
M 2.0994880318077413e+20
deltau 1.01403022957

M 2.1405783427612554e+20
deltau 1.0074973741014116e-23
M 2.140751385308117e+20
deltau 1.0074702167134165e-23
M 2.1409244371798183e+20
deltau 1.0074430607886517e-23
M 2.1410974983766098e+20
deltau 1.0074159063269996e-23
M 2.1412705688987432e+20
deltau 1.0073887533283407e-23
M 2.141443648746469e+20
deltau 1.0073616017925585e-23
M 2.141616737920039e+20
deltau 1.0073344517195334e-23
M 2.141789836419704e+20
deltau 1.0073073031091473e-23
M 2.1419629442457154e+20
deltau 1.0072801559612826e-23
M 2.142136061398324e+20
deltau 1.0072530102758205e-23
M 2.1423091878777812e+20
deltau 1.0072258660526435e-23
M 2.142482323684338e+20
deltau 1.0071987232916322e-23
M 2.1426554688182456e+20
deltau 1.0071715819926699e-23
M 2.142828623279755e+20
deltau 1.0071444421556376e-23
M 2.143001787069118e+20
deltau 1.0071173037804173e-23
M 2.143174960186585e+20
deltau 1.0070901668668904e-23
M 2.1433481426324075e+20
deltau 1.0070630314149396e-23
M 2.1435213344068367e+20
deltau 1.007035897424447e-23
M 2.1436945355101235

deltau 1.0005389461538937e-23
M 2.185708633452008e+20
deltau 1.0005121623939237e-23
M 2.185884099523458e+20
deltau 1.0004853800670865e-23
M 2.186059574984795e+20
deltau 1.0004585991732676e-23
M 2.186235059836271e+20
deltau 1.000431819712352e-23
M 2.1864105540781366e+20
deltau 1.0004050416842243e-23
M 2.1865860577106428e+20
deltau 1.0003782650887693e-23
M 2.186761570734041e+20
deltau 1.000351489925873e-23
M 2.1869370931485822e+20
deltau 1.0003247161954191e-23
M 2.1871126249545177e+20
deltau 1.0002979438972934e-23
M 2.1872881661520986e+20
deltau 1.0002711730313808e-23
M 2.1874637167415758e+20
deltau 1.0002444035975664e-23
M 2.1876392767232007e+20
deltau 1.0002176355957357e-23
M 2.1878148460972245e+20
deltau 1.0001908690257727e-23
M 2.1879904248638983e+20
deltau 1.0001641038875632e-23
M 2.1881660130234732e+20
deltau 1.0001373401809924e-23
M 2.1883416105762003e+20
deltau 1.0001105779059452e-23
M 2.1885172175223308e+20
deltau 1.0000838170623066e-23
M 2.188692833862116e+20
deltau 1.000057057